In [28]:
import librosa
import os, gc
import numpy as np
import random

from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, accuracy_score

import xgboost as xgb

In [3]:
walk_dir = "genres"

Y = {'metal': 0, 'disco': 1, 'classical': 2, 'hiphop': 3, 'jazz': 4,
     'country': 5, 'pop': 6, 'blues': 7, 'reggae': 8, 'rock': 9}

all_input_data = []
all_input_class = []
for root, subdirs, files in os.walk(walk_dir):
    print("Walk on dir:",root)
    for filename in files:
        if filename.endswith('.au'):
            file_path = os.path.join(root, filename)
            ppFileName = file_path.replace(".au",".pp")
            try:
                signal, fs = librosa.load(file_path, offset=12, duration=6)
                mfccs = librosa.feature.mfcc(signal, sr=fs, n_mfcc=12)
                l = mfccs.shape[1]
                mfccs = mfccs[:, int(0.2 * l):int(0.8 * l)]

                mean = mfccs.mean(axis=1)
                covar = np.cov(mfccs, rowvar=1)

                mean.resize(1, mean.shape[0])
                # it returns matrix.. not useful for machine learning algorithms except KNN
                npArray = np.concatenate((mean, covar), axis=0)
                templist = []
                for ls in np.nditer(npArray):
                    templist.append(ls)
                    
                all_input_data.append(templist)
                all_input_class.append(Y[filename.split('.')[0]])

            except Exception as e:
                print ("Error accured" + str(e))

Walk on dir: genres
Walk on dir: genres/pop
Walk on dir: genres/blues
Walk on dir: genres/classical
Walk on dir: genres/country
Walk on dir: genres/hiphop
Walk on dir: genres/rock
Walk on dir: genres/reggae
Walk on dir: genres/metal
Walk on dir: genres/disco
Walk on dir: genres/jazz


### Shuffling all data

In [4]:
%%time
print("Shuffle input data to get better results")

c = list(zip(all_input_data, all_input_class))
random.shuffle(c)
all_input_shuffledData, all_input_shuffledClass= zip(*c)

print("->all_input_shuffledData:",len(all_input_shuffledData))
print("->all_input_shuffledClass:",len(all_input_class))

Shuffle input data to get better results
->all_input_shuffledData: 1000
->all_input_shuffledClass: 1000
CPU times: user 6.7 ms, sys: 9.39 ms, total: 16.1 ms
Wall time: 5.2 ms


### KNN

In [5]:
%%time
kf = KFold(n_splits=5)

for trainIndex, testIndex in kf.split(all_input_data):
    X_train, X_test = [all_input_shuffledData[i] for i in trainIndex], [all_input_shuffledData[i] for i in testIndex]
    y_train, y_test = [all_input_shuffledClass[i] for i in trainIndex], [all_input_shuffledClass[i] for i in testIndex]
    
    knn = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2,metric='minkowski', metric_params=None, n_jobs=1)
    knn.fit(X_train,y_train)
    k_score = knn.score(X_test,y_test)
    print("KNN Score: ", k_score)

KNN Score:  0.295
KNN Score:  0.39
KNN Score:  0.385
KNN Score:  0.36
KNN Score:  0.335
CPU times: user 247 ms, sys: 1.3 ms, total: 248 ms
Wall time: 245 ms


### SVM

In [6]:
%%time
kf = KFold(n_splits=5)

for trainIndex, testIndex in kf.split(all_input_data):
    X_train, X_test = [all_input_shuffledData[i] for i in trainIndex], [all_input_shuffledData[i] for i in testIndex]
    y_train, y_test = [all_input_shuffledClass[i] for i in trainIndex], [all_input_shuffledClass[i] for i in testIndex]
    
    svm1 = svm.LinearSVC(C=10, loss='squared_hinge', penalty='l2', tol=0.00001)
    svm1.fit(X_train,y_train)
    scoreSVM = svm1.score(X_test, y_test)
    print("Support Vector Machines: ", scoreSVM)   
   

Support Vector Machines:  0.47
Support Vector Machines:  0.445
Support Vector Machines:  0.43
Support Vector Machines:  0.48
Support Vector Machines:  0.45
CPU times: user 8.25 s, sys: 8.49 ms, total: 8.26 s
Wall time: 8.26 s


### Decision Tree

In [7]:
%%time
kf = KFold(n_splits=5)

for trainIndex, testIndex in kf.split(all_input_data):
    X_train, X_test = [all_input_shuffledData[i] for i in trainIndex], [all_input_shuffledData[i] for i in testIndex]
    y_train, y_test = [all_input_shuffledClass[i] for i in trainIndex], [all_input_shuffledClass[i] for i in testIndex]
    
    regr_1 = DecisionTreeRegressor(max_depth=6)
    regr_2 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=6), n_estimators=300)
    regr_1.fit(X_train, y_train)
    regr_2.fit(X_train, y_train)
    y_1 = regr_1.predict(X_test)
    y_2 = regr_2.predict(X_test)
    score1 = regr_1.score(X_test, y_test)
    score2 = regr_2.score(X_test, y_test)
    print("Decision tree score1: ", score1, "Adaboost score2: ", score2)


Decision tree score1:  -0.15194172856058752 Adaboost score2:  0.13337219736576578
Decision tree score1:  -0.3022702917376032 Adaboost score2:  0.23997628487530753
Decision tree score1:  -0.12973617207173072 Adaboost score2:  0.27200884597091957
Decision tree score1:  0.058347116760149365 Adaboost score2:  0.29419789609337654
Decision tree score1:  -0.021905785158032876 Adaboost score2:  0.2960323341508614
CPU times: user 42.2 s, sys: 335 µs, total: 42.2 s
Wall time: 42.2 s


### XGBoost

In [27]:
kf = KFold(n_splits=5)

for trainIndex, testIndex in kf.split(all_input_data):
    X_train, X_test = [all_input_shuffledData[i] for i in trainIndex], [all_input_shuffledData[i] for i in testIndex]
    y_train, y_test = [all_input_shuffledClass[i] for i in trainIndex], [all_input_shuffledClass[i] for i in testIndex]

    xgb_params = {
        'eta': 0.05,
        'max_depth': 5,
        'subsample': 0.9,
        'objective': 'multi:softmax',
        'silent': 1,
        'colsample_bytree': 0.9,
        'num_class':10
    }

    d_train = xgb.DMatrix(X_train, y_train)
    d_valid = xgb.DMatrix(X_test, y_test)
    
    model = xgb.train(xgb_params, d_train, num_boost_round = 240)
    pred_prob = model.predict(d_valid)
    print('Accuracy Score:',accuracy_score(y_test, pred_prob))


Accuracy Score: 0.615
Accuracy Score: 0.63
Accuracy Score: 0.58
Accuracy Score: 0.64
Accuracy Score: 0.68


In [29]:
# run garbage collector manually
gc.collect()

1715